## Memory

In [ ]:
## Simple Example: save all conversations

In [ ]:
!pip install "langchain==0.3.27"

In [ ]:
# no need of output parser since the output of the conversationchain is already string
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

llm = ChatOllama(model="phi")

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
result = conversation.invoke("where is the captial of Portugal?")       # print result to see it, it's a dictionary.
print(result['response'])




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: where is the captial of Portugal?
AI:

> Finished chain.
 Lisbon is the capital city of Portugal. It's located on the Tagus River, which makes it easy for ships and boats to come in and out. Would you like to know more about Lisbon?



In [ ]:
# no need of output parser since the output of the conversationchain is already string
# if we want to use the prompt, no need to use ConversationChain, use RunnableWithMessageHistory
# RunnableWithMessageHistory: use session id to save the conversation + handle multiple sessions (users) + custom prompt

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableWithMessageHistory

llm = ChatOllama(model="phi")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an Assistant. Answer questions very breifly"),
        MessagesPlaceholder(variable_name="history"),                        # history of previous messages injected to the new message
        ("human", "{input}")
    ]
)

memory = ConversationBufferMemory(return_messages=True)    # save messages as list of messages (not string)


chain = RunnableWithMessageHistory(
    prompt | llm,
    lambda session_id: memory,          # how to get memory
    input_messages_key="input",         # where new user message is
    history_messages_key="history"      # where chat history goes
)


chain.invoke(
    {"input": "Where is the capital of Portugal?"},
    config={"configurable": {"session_id": "test"}}
)

## Summary

note 1: conversation | StrOutputParser() is wrong, since the output of the conversation chain is a string and the input of the parser should be a message
note 2: default prompt > use ConversationChain, custom prompt > use RunnableWithMessageHistory

    